<a href="https://colab.research.google.com/github/Dash400air/CommonLit/blob/main/commonlit_Roberta%2BLightGBM%2BOptuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers==4.5.0 pytorch-lightning==1.2.7

     |████████████████████████████████| 2.1 MB 35.4 MB/s 
     |████████████████████████████████| 830 kB 60.0 MB/s 
     |████████████████████████████████| 895 kB 44.0 MB/s 
     |████████████████████████████████| 3.3 MB 41.4 MB/s 
     |████████████████████████████████| 269 kB 63.1 MB/s 
     |████████████████████████████████| 829 kB 52.6 MB/s 
     |████████████████████████████████| 118 kB 69.1 MB/s 
     |████████████████████████████████| 234 kB 75.1 MB/s 
     |████████████████████████████████| 1.3 MB 44.0 MB/s 
     |████████████████████████████████| 294 kB 65.4 MB/s 
     |████████████████████████████████| 142 kB 72.6 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=689a370134e6acccd5e934cff1f6ad9bdeb4cf5ed00153ba32e37aeedb5da970
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44636 sha256=05e6ee4

In [3]:
!pip install nltk

In [4]:
!pip install textstat

     |████████████████████████████████| 99 kB 9.7 MB/s 
     |████████████████████████████████| 2.0 MB 43.7 MB/s 


In [5]:
!pip install optuna

     |████████████████████████████████| 301 kB 32.0 MB/s 
     |████████████████████████████████| 80 kB 10.7 MB/s 
     |████████████████████████████████| 164 kB 70.0 MB/s 
     |████████████████████████████████| 75 kB 5.5 MB/s 
     |████████████████████████████████| 111 kB 73.2 MB/s 
     |████████████████████████████████| 141 kB 65.7 MB/s 
     |████████████████████████████████| 49 kB 7.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=b946a090fcd90fa1f01d4efec4a40d56881770fb167ed146978beef708da6a58
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [6]:
import os
import pandas as pd
import numpy as np


import random
import glob
import re
import string
from tqdm import tqdm
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import RobertaTokenizer, RobertaModel
import pytorch_lightning as pl

from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.preprocessing.text import Tokenizer

import lightgbm as lgb
import optuna.integration.lightgbm as lgb_o

from fastprogress.fastprogress import  progress_bar

In [7]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Common Lit kaggle/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Common Lit kaggle/test.csv')

# Textstat

In [8]:
import textstat

def get_readability(df):
    excerpt = df.excerpt
    excerpt_list = excerpt.values.tolist()
    readability_list = []

    for text in excerpt_list: 
        fre = textstat.flesch_reading_ease(text)
        smog = textstat.smog_index(text)
        fkg = textstat.flesch_kincaid_grade(text)
        cli = textstat.coleman_liau_index(text)
        arbi = textstat.automated_readability_index(text)
        dcrs = textstat.dale_chall_readability_score(text)
        dw = textstat.difficult_words(text)
        lwf = textstat.linsear_write_formula(text)
        gf = textstat.gunning_fog(text)

        readability_list.append([fre, smog, fkg, cli, arbi, dcrs, dw, lwf, gf])
    readability = pd.DataFrame(readability_list)
    readability = readability.rename(columns={0: 'fre',
                                             1: 'smog',
                                             2: 'fkg',
                                             3: 'cli',
                                             4: 'arbi',
                                             5: 'dcrs',
                                             6: 'dw',
                                             7: 'lwf',
                                             8: 'gf'
                                             }
                                    )
    return readability

In [9]:
readability_train = get_readability(df)
readability_test = get_readability(test_df)

In [10]:
readability_train.head()

,fre,smog,fkg,cli,arbi,dcrs,dw,lwf,gf
0,80.31,8.6,6.1,8.06,8.3,6.65,25,9.000000,8.31
1,82.54,8.3,5.2,6.78,7.2,5.92,17,7.285714,7.53
2,75.74,10.1,7.9,7.20,10.1,6.29,17,14.750000,10.49
3,72.02,6.7,11.4,8.54,16.4,6.61,14,12.500000,13.61
4,75.47,8.8,10.0,4.83,11.8,1.57,1,13.500000,11.76


# Preprocessing

In [11]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [12]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
more_stopwords = ['u', 'im', 'c']
stop_words = stop_words + more_stopwords

def remove_stopwords(text):
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
stemmer = nltk.SnowballStemmer("english")

def stemm_text(text):
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    return text

In [14]:
def preprocess_data(text, strip=False):
    # Clean puntuation, urls, and so on
    text = clean_text(text)
    # Remove stopwords
    #text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    # Stemm all the words in the sentence
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    
    if strip:
        text = text.strip()
    
    return text

In [15]:
df['excerpt'] = df['excerpt'].apply(preprocess_data)
df.head()

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,when the young peopl return to the ballroom it...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,all through dinner time mrs fayr was somewhat ...,-0.315372,0.480805
2,b69ac6792,NaN,NaN,as roger had predict the snow depart as quick ...,-0.580118,0.476676
3,dd1000b26,NaN,NaN,and outsid befor the palac a great garden was ...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,onc upon a time there were three bear who live...,0.247197,0.510845


# Bert Dataset

In [16]:
class BertDataset(nn.Module):
    def __init__(self, df, tokenizer, max_len=128):
        self.excerpt = df['excerpt'].to_numpy()
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(
            self.excerpt[idx],
            return_tensors='pt',
            max_length=self.max_len,
            padding='max_length',
            truncation=True
        )
        return encode
    
    def __len__(self):
        return len(self.excerpt)
    

def get_embeddings(df, path, plot_losses=True, verbose=True):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"{device} is used")
            
    MODEL_PATH = path
    model = RobertaModel.from_pretrained(MODEL_PATH, num_labels=1)
    tokenizer = RobertaTokenizer.from_pretrained(MODEL_PATH)
    model.to(device)
    model.eval()

    ds = BertDataset(df, tokenizer, config['max_len'])
    dl = DataLoader(
        ds,
        batch_size=config["batch_size"],
        shuffle=False,
        num_workers = 4,
        pin_memory=True,
        drop_last=False
    )

    embeddings = list()
    with torch.no_grad():
        for i, inputs in progress_bar(list(enumerate(dl))):
            inputs = {key:val.reshape(val.shape[0], -1).to(device) for key, val in inputs.items()}
            outputs = model(**inputs)
            outputs = outputs[0][:, 0].detach().cpu().numpy()
            embeddings.extend(outputs)
            
    return np.array(embeddings)

In [17]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [18]:
config = {
    'batch_size': 128,
    'max_len': 256,
    'seed': 42,
}
seed_everything(seed=config['seed'])

train_embeddings =  get_embeddings(df,'roberta-large')
test_embeddings = get_embeddings(test_df,'roberta-large')

cuda is used


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



cuda is used


# Tf-idf, Wordcounts

In [19]:
vectorizer = TfidfVectorizer(max_features=1000)

train_word_tf_idfs = vectorizer.fit_transform(df['excerpt'].values).toarray()
train_word_tf_idfs = pd.DataFrame(train_word_tf_idfs)
train_word_tf_idfs.columns = vectorizer.get_feature_names()

test_word_tf_idfs = vectorizer.fit_transform(test_df['excerpt'].values).toarray()
test_word_tf_idfs = pd.DataFrame(test_word_tf_idfs)
test_word_tf_idfs.columns = vectorizer.get_feature_names()

def count_words(df):
    word_counts = []
    for sentence in progress_bar(df['excerpt']):
        sentence = re.sub("[^a-zA-Z]", " ", sentence)
        
        words = sentence.split()
        count = len(words)
        word_counts.append(count)
        
    return word_counts

df['count_words'] = count_words(df)
test_df['count_words'] = count_words(test_df)

# Text matrix

In [20]:
M_train, M_test = df['excerpt'], test_df['excerpt']

texts_train, texts_test = M_train, M_test
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_train)
tokenizer.fit_on_texts(texts_test)

matrix1 = tokenizer.texts_to_matrix(texts_train, "binary")
matrix2 = tokenizer.texts_to_matrix(texts_train, "count")
matrix3 = tokenizer.texts_to_matrix(texts_train, "tfidf")
M1_train = pd.DataFrame(matrix1)
M2_train = pd.DataFrame(matrix2)
M3_train = pd.DataFrame(matrix3)

matrix1 = tokenizer.texts_to_matrix(texts_test, "binary")
matrix2 = tokenizer.texts_to_matrix(texts_test, "count")
matrix3 = tokenizer.texts_to_matrix(texts_test, "tfidf")
M1_test = pd.DataFrame(matrix1)
M2_test = pd.DataFrame(matrix2)
M3_test = pd.DataFrame(matrix3)

# Get Dataframe

In [21]:
X_train = pd.DataFrame(train_embeddings)
X_train = pd.concat([X_train, readability_train[['fre', 'smog', 'dcrs', 'dw']], df['count_words'], M2_train], axis=1)

X_test = pd.DataFrame(test_embeddings)
X_test = pd.concat([X_test, readability_test[['fre', 'smog', 'dcrs', 'dw']], test_df['count_words'], M2_test], axis=1)

train_columns = X_train.columns.tolist()
test_columns = X_test.columns.tolist()

#diff_columns = list(set(test_columns) - set(train_columns))
#X_train = X_train.reindex(columns = train_columns + diff_columns)

#diff_columns = list(set(train_columns) - set(test_columns))
#X_test = X_test.reindex(columns = test_columns + diff_columns)

In [22]:
y_train = df['target']

In [23]:
train = pd.concat([X_train, y_train], axis=1)

# K-fold

In [24]:
def create_folds(data, num_splits):
    # we create a new column called kfold and fill it with -1
    data["kfold"] = -1
    
    # the next step is to randomize the rows of the data
    data = data.sample(frac=1).reset_index(drop=True)

    # calculate number of bins by Sturge's rule
    # I take the floor of the value, you can also
    # just round it
    num_bins = int(np.floor(1 + np.log2(len(data))))
    
    # bin targets
    data.loc[:, "bins"] = pd.cut(
        data["target"], bins=num_bins, labels=False
    )
    
    # initiate the kfold class from model_selection module
    kf = StratifiedKFold(n_splits=num_splits)
    
    # fill the new kfold column
    # note that, instead of targets, we use bins!
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    
    # drop the bins column
    data = data.drop("bins", axis=1)

    # return dataframe with folds
    return data


# create folds
train = create_folds(train, num_splits=5)

In [25]:
y_train = train[['target', 'kfold']]
X_train = train.drop(['target'], axis=1)

# Optuna

In [31]:
params = {
    'metric': 'rmse',
    'objective': 'regression',
    'seed': 42, #
}

fold = 0
x_tr = X_train[X_train.kfold != fold].reset_index(drop=True).drop(['kfold'], axis=1)
x_va = X_train[X_train.kfold == fold].reset_index(drop=True).drop(['kfold'], axis=1)
y_tr = y_train[y_train.kfold != fold].reset_index(drop=True).drop(['kfold'], axis=1)
y_va = y_train[y_train.kfold == fold].reset_index(drop=True).drop(['kfold'], axis=1)

train_set = lgb.Dataset(x_tr, y_tr)
val_set = lgb.Dataset(x_va, y_va, reference=train_set)

gbm = lgb_o.train(params,
                  train_set,
                  valid_sets=[train_set, val_set],
                  num_boost_round=10000,
                  early_stopping_rounds=100,
                  )

[I 2021-07-23 05:57:19,915] A new study created in memory with name: no-name-6429e7a5-2768-4538-8f6a-4b178a4cf6da

feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[1]	valid_0's rmse: 0.975735	valid_1's rmse: 0.98421
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 0.926405	valid_1's rmse: 0.944061
[3]	valid_0's rmse: 0.883451	valid_1's rmse: 0.913699
[4]	valid_0's rmse: 0.842041	valid_1's rmse: 0.886231
[5]	valid_0's rmse: 0.803598	valid_1's rmse: 0.861478
[6]	valid_0's rmse: 0.768168	valid_1's rmse: 0.836383
[7]	valid_0's rmse: 0.736386	valid_1's rmse: 0.815475
[8]	valid_0's rmse: 0.708406	valid_1's rmse: 0.797906
[9]	valid_0's rmse: 0.681881	valid_1's rmse: 0.782084
[10]	valid_0's rmse: 0.657974	valid_1's rmse: 0.769365
[11]	valid_0's rmse: 0.636407	valid_1's rmse: 0.757025
[12]	valid_0's rmse: 0.615456	valid_1's rmse: 0.747995
[13]	valid_0's rmse: 0.596277	valid_1's rmse: 0.739818
[14]	valid_0's rmse: 0.578771	valid_1's rmse: 0.73316
[15]	valid_0's rmse: 0.562197	valid_1's rmse: 0.727224
[16]	valid_0's rmse: 0.546513	valid_1's rmse: 0.719157
[17]	valid_0's rmse: 0.531543	valid_1's rmse: 0.713406
[18]	valid_0'


feature_fraction, val_score: 0.647460:  14%|#4        | 1/7 [00:54<05:24, 54.11s/it][I 2021-07-23 05:58:14,060] Trial 0 finished with value: 0.6474599404197828 and parameters: {'feature_fraction': 0.4}. Best is trial 0 with value: 0.6474599404197828.

feature_fraction, val_score: 0.647460:  14%|#4        | 1/7 [00:54<05:24, 54.11s/it]

[1]	valid_0's rmse: 0.975518	valid_1's rmse: 0.983634
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 0.922089	valid_1's rmse: 0.945819
[3]	valid_0's rmse: 0.876007	valid_1's rmse: 0.913308
[4]	valid_0's rmse: 0.834413	valid_1's rmse: 0.883853
[5]	valid_0's rmse: 0.796446	valid_1's rmse: 0.859641
[6]	valid_0's rmse: 0.761538	valid_1's rmse: 0.835983
[7]	valid_0's rmse: 0.730368	valid_1's rmse: 0.816058
[8]	valid_0's rmse: 0.702144	valid_1's rmse: 0.800327
[9]	valid_0's rmse: 0.675476	valid_1's rmse: 0.78487
[10]	valid_0's rmse: 0.651743	valid_1's rmse: 0.773694
[11]	valid_0's rmse: 0.629913	valid_1's rmse: 0.762806
[12]	valid_0's rmse: 0.609331	valid_1's rmse: 0.752296
[13]	valid_0's rmse: 0.590561	valid_1's rmse: 0.744361
[14]	valid_0's rmse: 0.572451	valid_1's rmse: 0.737991
[15]	valid_0's rmse: 0.555679	valid_1's rmse: 0.732309
[16]	valid_0's rmse: 0.540057	valid_1's rmse: 0.725733
[17]	valid_0's rmse: 0.52578	valid_1's rmse: 0.719692
[18]	valid_0'


feature_fraction, val_score: 0.647460:  29%|##8       | 2/7 [01:41<04:20, 52.02s/it][I 2021-07-23 05:59:01,180] Trial 1 finished with value: 0.6566563997311972 and parameters: {'feature_fraction': 0.5}. Best is trial 0 with value: 0.6474599404197828.



...

[571]	valid_0's rmse: 0.215036	valid_1's rmse: 0.630127
[572]	valid_0's rmse: 0.214735	valid_1's rmse: 0.630178
[573]	valid_0's rmse: 0.214526	valid_1's rmse: 0.63002
[574]	valid_0's rmse: 0.21412	valid_1's rmse: 0.630222
[575]	valid_0's rmse: 0.213835	valid_1's rmse: 0.630286
[576]	valid_0's rmse: 0.213574	valid_1's rmse: 0.630091
[577]	valid_0's rmse: 0.213283	valid_1's rmse: 0.630326
[578]	valid_0's rmse: 0.212929	valid_1's rmse: 0.630496
[579]	valid_0's rmse: 0.212552	valid_1's rmse: 0.630667
[580]	valid_0's rmse: 0.212203	valid_1's rmse: 0.630607
[581]	valid_0's rmse: 0.211882	valid_1's rmse: 0.630548
[582]	valid_0's rmse: 0.211614	valid_1's rmse: 0.630521
Early stopping, best iteration is:
[482]	valid_0's rmse: 0.244435	valid_1's rmse: 0.627763


min_data_in_leaf, val_score: 0.618772:  80%|########  | 4/5 [00:56<00:14, 14.48s/it][I 2021-07-23 06:33:38,273] Trial 63 finished with value: 0.6277634812453391 and parameters: {'min_child_samples': 50}. Best is trial 63 with value: 0.6277634812453391.

min_data_in_leaf, val_score: 0.618772:  80%|########  | 4/5 [00:56<00:14, 14.48s/it]

[1]	valid_0's rmse: 0.993296	valid_1's rmse: 0.995211
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's rmse: 0.960029	valid_1's rmse: 0.963804
[3]	valid_0's rmse: 0.932775	valid_1's rmse: 0.938035
[4]	valid_0's rmse: 0.902567	valid_1's rmse: 0.912836
[5]	valid_0's rmse: 0.880825	valid_1's rmse: 0.893748
[6]	valid_0's rmse: 0.859247	valid_1's rmse: 0.874842
[7]	valid_0's rmse: 0.838669	valid_1's rmse: 0.857286
[8]	valid_0's rmse: 0.819888	valid_1's rmse: 0.840539
[9]	valid_0's rmse: 0.803376	valid_1's rmse: 0.825891
[10]	valid_0's rmse: 0.788731	valid_1's rmse: 0.813092
[11]	valid_0's rmse: 0.775088	valid_1's rmse: 0.801779
[12]	valid_0's rmse: 0.762818	valid_1's rmse: 0.791073
[13]	valid_0's rmse: 0.750592	valid_1's rmse: 0.780705
[14]	valid_0's rmse: 0.740994	valid_1's rmse: 0.772338
[15]	valid_0's rmse: 0.73049	valid_1's rmse: 0.761852
[16]	valid_0's rmse: 0.720814	valid_1's rmse: 0.753
[17]	valid_0's rmse: 0.71231	valid_1's rmse: 0.746257
[18]	valid_0's r


min_data_in_leaf, val_score: 0.618772: 100%|##########| 5/5 [01:07<00:00, 13.58s/it]

[401]	valid_0's rmse: 0.289783	valid_1's rmse: 0.634856
[402]	valid_0's rmse: 0.289321	valid_1's rmse: 0.634895
[403]	valid_0's rmse: 0.289005	valid_1's rmse: 0.635039
[404]	valid_0's rmse: 0.288525	valid_1's rmse: 0.635026
[405]	valid_0's rmse: 0.287981	valid_1's rmse: 0.635039
Early stopping, best iteration is:
[305]	valid_0's rmse: 0.338981	valid_1's rmse: 0.632734


# Run Model

In [35]:
params = {
    'boosting_type': 'gbdt', #
    'metric': 'rmse',
    'objective': 'regression',
    'seed': 42, #
    'learning_rate': 2e-5, #
    "n_jobs": -1,
    'max_depth': 2, #
    "verbose": -1
}

best_params = gbm.params

pred = np.zeros(X_test.shape[0])
rmses = []

for fold in range(5):
    x_tr = X_train[X_train.kfold != fold].reset_index(drop=True).drop(['kfold'], axis=1)
    x_va = X_train[X_train.kfold == fold].reset_index(drop=True).drop(['kfold'], axis=1)
    y_tr = y_train[y_train.kfold != fold].reset_index(drop=True).drop(['kfold'], axis=1)
    y_va = y_train[y_train.kfold == fold].reset_index(drop=True).drop(['kfold'], axis=1)

    train_set = lgb.Dataset(x_tr, y_tr)
    val_set = lgb.Dataset(x_va, y_va, reference=train_set)

    model = lgb.train(
        best_params,
        train_set, 
        num_boost_round=10000,
        early_stopping_rounds=100,
        valid_sets=[train_set, val_set], 
        verbose_eval=-1
    )

    y_pred = model.predict(x_va)
    rmse = np.sqrt(mse(y_va, y_pred))
    rmses.append(rmse)
    
    tmp_pred = model.predict(X_test)
    pred += tmp_pred / 5
    
print("\n", "Mean Fold RMSE:", np.mean(rmses))

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[515]	training's rmse: 0.23288	valid_1's rmse: 0.618772
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[439]	training's rmse: 0.260649	valid_1's rmse: 0.629667
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[154]	training's rmse: 0.443728	valid_1's rmse: 0.625282
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[479]	training's rmse: 0.242453	valid_1's rmse: 0.666938
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[363]	training's rmse: 0.295982	valid_1's rmse: 0.644118

 Mean Fold RMSE: 0.6369553771092951
